<a href="https://colab.research.google.com/github/JeanLuc-Oudshoorn/Net_Merchandise_Value/blob/master/nmv_model_in_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a machine learning model to forecast net merchandise value

In this notebook we will be building an adaptive model to forecast NMV. In order to do so, we have to explode the data which will result in a very large input dataframe. Therefore the PySpark API will be used to train the model.

### Installing PySpark on Colab

A few commands are needed to install and activate PySpark on Colab. This should only take about a minute.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()

If PySpark is successfully loaded in the environment we can now create a spark session. We can give it a name and add external repositories and packages with the 'config' option. The command below loads the PySpark implementation for the LightGBM algorithm.

In [4]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("NMV Model")\
                                        .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.11.0")\
                                        .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")\
                                        .getOrCreate()

We can now check if the spark session is build correctly. In this case it will only be a local session, so we could also have used pandas. In reality, the application would be run on a Hadoop cluster.

In [5]:
spark

### Loading the simulated data

For this example we will be using a *simulated* data set, specifically developed for this training. Because the data is fake, it might contain some quirks that you would not see in real data (e.g.: strange color & product group combinations). We can read the file directly from the GitHub link and add it to the spark session.

In [7]:
from pyspark import SparkFiles
url = 'https://raw.githubusercontent.com/JeanLuc-Oudshoorn/Net_Merchandise_Value/master/fashion_data_simulation.csv'

spark.sparkContext.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("fashion_data_simulation.csv"), header=True, inferSchema= True)

The data looks as follows. In a real NMV project it will most likely be required to load historical orders and product information as two separate tables and join them togheter. For sake of simplicity, this step is skipped here.

In [8]:
df.show()

+------+-------------+--------+-----------+-------------+-------------+----+---------------+------------+-------+-----+-----------+----------+-----------+-------------+
| color|        brand|order_id|customer_id|product_group|product_class|size|sold_to_country|prod_country|    fit|price|is_returned|order_date|return_date|item_trans_id|
+------+-------------+--------+-----------+-------------+-------------+----+---------------+------------+-------+-----+-----------+----------+-----------+-------------+
|  Blue|         Zara|     101|       7884|         Polo|       Casual|   L|          Spain|  Bangladesh|Regular|  180|          1|2021-08-01| 2021-08-12|            0|
| Black|         Zara|     101|       7884|       Loafer|         null|   S|        Denmark|    Portugal|   Slim|  174|          1|2021-08-01| 2021-08-10|            1|
| White|         Zara|     431|      59945|      Handbag|  Accessories|null|          Spain|      Turkey|   null|  227|          0|2021-08-01|       null| 

### Feature Engineering

#### Date Features
Clearly, there are some opportunities for feature engineering in this data. For example, the order date cannot be used in its current form. Instead we can derive new predictive features from the order date.

In [10]:
from pyspark.sql import functions as F

# Order day of year
df = df.withColumn('day_of_year', F.dayofyear(F.col('order_date')))

# Order year
df = df.withColumn('year', F.year(F.col('order_date')))

# Order month
df = df.withColumn('month', F.month(F.col('order_date')))

# Order week of year
df = df.withColumn('week_of_year', F.weekofyear(F.col('order_date')))

We can now check to see if the new features were created properly.

In [11]:
df.show()

+------+-------------+--------+-----------+-------------+-------------+----+---------------+------------+-------+-----+-----------+----------+-----------+-------------+-----------+----+-----+------------+
| color|        brand|order_id|customer_id|product_group|product_class|size|sold_to_country|prod_country|    fit|price|is_returned|order_date|return_date|item_trans_id|day_of_year|year|month|week_of_year|
+------+-------------+--------+-----------+-------------+-------------+----+---------------+------------+-------+-----+-----------+----------+-----------+-------------+-----------+----+-----+------------+
|  Blue|         Zara|     101|       7884|         Polo|       Casual|   L|          Spain|  Bangladesh|Regular|  180|          1|2021-08-01| 2021-08-12|            0|        213|2021|    8|          30|
| Black|         Zara|     101|       7884|       Loafer|         null|   S|        Denmark|    Portugal|   Slim|  174|          1|2021-08-01| 2021-08-10|            1|        213|

#### Geometric Transformations
One-hot encoding the day of year will lead to many new columns in the data. Below is an example of how to geometrically transform this feature.

In [12]:
import numpy as np

# Sine base function
def sin_base(x, period=365.25):
    return float(np.sin(x / period * 2 * np.pi))

# Cosine base function
def cos_base(x, period=365.25):
    return float(np.cos(x / period * 2 * np.pi))

# Convert into PySpark function
sin_transformer = F.udf(lambda x:sin_base(x)) 

# Convert into PySpark function
cos_transformer = F.udf(lambda x:cos_base(x)) 

After creating the geometric transformers we only have to apply them to the correct column.

In [13]:
# Apply transformation
df = df.withColumn('sin_order_date', sin_transformer(F.col('day_of_year')))
df = df.withColumn('cos_order_date', cos_transformer(F.col('day_of_year')))

In [17]:
df.select('order_date', 'sin_order_date', 'cos_order_date').distinct().orderBy('order_date').show()

+----------+--------------------+-------------------+
|order_date|      sin_order_date|     cos_order_date|
+----------+--------------------+-------------------+
|2021-08-01|-0.49906860268853154|-0.8665624788845387|
|2021-08-02| -0.5139010013093099|-0.8578494977869303|
|2021-08-03| -0.5285813283559295|-0.8488826652214551|
|2021-08-04|  -0.543105239683432|-0.8396646346181323|
|2021-08-05| -0.5574684374327888|-0.8301981337405215|
|2021-08-06| -0.5716666713027055|-0.8204859638785325|
|2021-08-07| -0.5856957398073508|-0.8105309990194823|
|2021-08-08| -0.5995514915196486|-0.8003361849976326|
|2021-08-09| -0.6132298262997502|-0.7899045386224707|
|2021-08-10| -0.6267266965083301|-0.7792391467859886|
|2021-08-11| -0.6400381082043503|-0.7683431655492202|
|2021-08-12| -0.6531601223269297| -0.757219819208313|
|2021-08-13|  -0.666088855860977|-0.7458723993404063|
|2021-08-14| -0.6788204829862355| -0.734304263829602|
|2021-08-15| -0.6913512362094071| -0.722518835873311|
|2021-08-16| -0.703677407479

#### Order Features
Order features can be engineered by using a window and selectively partitioning on the desired features. Then we can count or sum over the window to create the feature that we want.

In [22]:
from pyspark.sql import types as T
from pyspark.sql import Window as psw

# Total number of items in one order
w = psw.orderBy('order_date').partitionBy('order_id') 
df = df.withColumn('n_item_same_order', F.count('item_trans_id').over(w))

# Number of items in the same product class in the same order
w = psw.orderBy('order_date').partitionBy('order_id', 'product_class')
df = df.withColumn('n_same_pc_same_order', F.count('item_trans_id').over(w))
df = df.fillna(0, subset=["n_same_pc_same_order"])

# Number of items in the same product group in the same order
w = psw.orderBy('order_date').partitionBy('order_id', 'product_group')
df = df.withColumn('n_same_pg_same_order', F.count('item_trans_id').over(w))
df = df.fillna(0, subset=["n_same_pg_same_order"])

# Total order value
w = psw.orderBy('order_date').partitionBy('order_id')
df = df.withColumn('order_value', F.sum('price').over(w))
df = df.fillna(0, subset=["order_value"])

#### Data Explosion
To make adaptive forecasts, the data needs to be exploded. One key assumption here is that customers will return all unwanted items at once. That is to say, if one of multiple items in an order is returned, we assume the rest is kept. We will also keep an unexploded version of the data because we will need this later for statistical features.

In [24]:
# Keep an unexploded version
df_order_unexpl = df.alias('df_unexpl')

In [27]:
import datetime as dt

# Transform
def explode_data(df_order,
                 return_window_size=33,
                ):
    """
    """

    # Create the purchase_order_return_date for all orders
    # Because if an item in a purchase order has been returned,
    # we will stop predicting for all items in this order

    w = psw.orderBy('order_date').partitionBy('order_id')
    df_order = (df_order
                .withColumn("is_purchase_order_returned", F.max('is_returned').over(w))
                .withColumn("purchase_order_return_date", F.min('return_date').over(w))
                .withColumn('purchase_order_return_date', F.when(F.col('purchase_order_return_date').isNull(),
                                                                 F.to_date(F.lit('2099-01-01'), 'yyyy-MM-dd'))
                            .otherwise(F.col('purchase_order_return_date')))
                )

    # Explode the data by the size of return window
    # Remove all the items in an order if one of the item has been returned

    df_order_expl = (
        df_order.withColumn('arr_days_after_order', F.array([F.lit(x) for x in range(return_window_size + 1)]))
            .withColumn("days_after_order", F.explode("arr_days_after_order"))
            .withColumn("current_date",
                        F.udf(lambda x, y: x + dt.timedelta(days=y), T.DateType())(F.col("order_date"),
                                                                                   F.col(
                                                                                       "days_after_order")))
            .filter(F.col("current_date") < F.col("purchase_order_return_date"))
    )

    return df_order_expl

Even though the return window is only 30 days, we assume a 33 day return window to account for shipping time.

In [28]:
df = explode_data(df, return_window_size = 33)

The result of the explosion is a data frame with many rows per item, with different values for 'days_after_order' and 'current_date'.

In [29]:
df.show()

+-----+-----+--------+-----------+-------------+-------------+----+---------------+------------+-------+-----+-----------+----------+-----------+-------------+-----------+----+-----+------------+-------------------+--------------------+-----------------+----------------------------+--------------------+--------------------+-----------+--------------------------+--------------------------+--------------------+----------------+------------+
|color|brand|order_id|customer_id|product_group|product_class|size|sold_to_country|prod_country|    fit|price|is_returned|order_date|return_date|item_trans_id|day_of_year|year|month|week_of_year|     sin_order_date|      cos_order_date|n_item_same_order|n_prod_same_ptype_same_order|n_same_pc_same_order|n_same_pg_same_order|order_value|is_purchase_order_returned|purchase_order_return_date|arr_days_after_order|days_after_order|current_date|
+-----+-----+--------+-----------+-------------+-------------+----+---------------+------------+-------+-----+----

The data has also become substantially larger (starting from about 130,000 rows). 

In [30]:
df.count()

2914060

#### Statistical Features
To calculate statistical features, such as the historical return rate of a product class, we again use window functions. However this time, we also use a date range because we cannot be sure if items will be returned or not if they have been ordered less than 30 days ago. 

In [34]:
def cal_hist_returns(df, suffix, keys, date_range, max_date):
    """
    Use a rolling window to calculate the no. of purchase, no. of returns and return rate for (-inf, -return_window) dates
    for the product group defined by keys.
    """
    df_rr_daily = (df
                   .groupBy(keys+['order_date'])
                   .agg({"is_returned": "sum", "item_trans_id": "count"})
                   .withColumn("n_days", F.datediff(F.lit(max_date), F.col('order_date')))
                  )

    w = psw.partitionBy(keys).orderBy(F.desc('n_days')).rangeBetween(*date_range)

    # Compute the rolling count of purchased and returned items for the last -return_window days
    df_rr_daily = (df_rr_daily
                   .withColumn(f'ret_{suffix}', F.sum('sum(is_returned)').over(w))
                   .withColumn(f'pur_{suffix}', F.sum('count(item_trans_id)').over(w))
                   .withColumn(f'rr_{suffix}', F.round(F.col(f'ret_{suffix}')/F.col(f'pur_{suffix}'), 3))
                   .select(keys + ['order_date', f'rr_{suffix}'])
                   .withColumnRenamed('order_date', 'current_date')
                  )
    
    return df_rr_daily

For this example we will be calculating the historical return rate per country, per product group. The output is a dataframe with the return rate for every unique combination of country and product group. As can be seen, the historical return rates must be calculated on an *unexploded* dataframe.

In [37]:
# Define keys for category and product type
keys_group = ['sold_to_country',  'product_group']

return_window = 33

date_range = (psw.unboundedPreceding, -return_window)

# - rr_cat_cnt_recent: return rate per country per category per gender
max_date = df.agg({"order_date": 'max'}).collect()[0]['max(order_date)']

df_rr_cat = cal_hist_returns(df_order_unexpl,
                             "group",
                             keys_group,
                             date_range,
                             max_date)

df_rr_cat.filter(df_rr_cat.rr_group.isNotNull()).show()

+---------------+-------------+------------+--------+
|sold_to_country|product_group|current_date|rr_group|
+---------------+-------------+------------+--------+
|        Austria|         Belt|  2021-09-04|     0.0|
|        Austria|         Belt|  2021-09-05|     0.0|
|        Austria|         Belt|  2021-09-07|   0.667|
|        Austria|         Belt|  2021-09-08|   0.571|
|        Austria|         Belt|  2021-09-09|     0.5|
|        Austria|         Belt|  2021-09-10|   0.444|
|        Austria|         Belt|  2021-09-11|   0.455|
|        Austria|         Belt|  2021-09-12|   0.455|
|        Austria|         Belt|  2021-09-13|     0.5|
|        Austria|         Belt|  2021-09-14|   0.462|
|        Austria|         Belt|  2021-09-18|   0.467|
|        Austria|         Belt|  2021-09-19|   0.467|
|        Austria|         Belt|  2021-09-20|   0.467|
|        Austria|         Belt|  2021-09-21|   0.467|
|        Austria|         Belt|  2021-09-22|     0.5|
|        Austria|         Be

Now we can join the historical return rates to our original data, fill missing values and fix non-sensical values.

In [38]:
 df = df.join(df_rr_cat, on=keys_group + ['current_date'], how='left')

 df = df.na.fill({'rr_group': 0.4})

 df = df.withColumn('rr_group', F.when(F.col('rr_group') < 0.09, 0.4)
                                 .otherwise(F.col('rr_group')))

#### Customer Features
Now for the most exciting and often most useful part: customer features. The first step is to calculate the historical return rate per customer, for every date. For this purpose we can use the same approach as calculating statistical features per product group.

In [39]:
keys_cust = ['customer_id']

recent_date_range = (psw.unboundedPreceding, -return_window)

keys_cust = ['customer_id']
df_rr_cust = cal_hist_returns(df_order_unexpl,
                              "cust",
                              keys_cust,
                              date_range,
                              max_date)

df = df.join(df_rr_cust, on=keys_cust + ['current_date'], how='left')

We can also easily calculate each customer's account age by calculating the number of days since the first purchase.

In [40]:
w = psw.orderBy('order_date').partitionBy('customer_id')

df = df.withColumn('cust_first_purchase_date', F.first('order_date').over(w))
df = df.withColumn('cust_account_age',
                           F.datediff(F.col('order_date'), F.col('cust_first_purchase_date')))

It is also useful to create a dummy column to indicate if this is the customers first order.

In [41]:
df = df.withColumn('is_first_order',
                  (F.col('cust_first_purchase_date') == F.col('order_date')).cast('int'))

We can also use window functions to create purchase- and return counts for for each customer. Here we don't have to limit the end date of the window to thirty days before, because we would like to know if a return was made even yesterday.

In [45]:
df_order_unexpl = df_order_unexpl.withColumn("n_days", F.datediff(F.lit(max_date), F.col('order_date')))

# Total number of returns made by the customer in past year
w = psw.partitionBy(["customer_id"]).orderBy(F.desc('n_days')).rangeBetween(-366, -1)

df_order_unexpl = (df_order_unexpl
                   .withColumn('pur_cust_before', F.count("item_trans_id").over(w))
                   .withColumn('ret_cust_before', F.sum("is_returned").over(w))
                  )

# Total number of returns made by the customer per product class in past year
w = psw.partitionBy(["customer_id", "product_class"]).orderBy(F.desc('n_days')).rangeBetween(-366, -1)

df_order_unexpl = (df_order_unexpl
                   .withColumn('pur_same_pc_cust_before', F.count("item_trans_id").over(w))
                   .withColumn('ret_same_pc_cust_before', F.sum("is_returned").over(w))
                   )


Finally, we can join the customer features back to our original data.

In [47]:
df = df.join(df_order_unexpl.select("customer_id", "order_date", "item_trans_id",
                                    'pur_cust_before',
                                    'ret_cust_before',
                                    'pur_same_pc_cust_before',
                                    'ret_same_pc_cust_before'),
                     
                     on=["customer_id", "order_date", "item_trans_id"], how="left")


Due to the lazy execution, the commands above will only be executed once a command such as 'show' or 'count' is called.

In [48]:
df.show()

+-----------+----------+-------------+------------+---------------+-------------+-----+-----+--------+-------------+----+------------+----+-----+-----------+-----------+-----------+----+-----+------------+--------------------+-------------------+-----------------+----------------------------+--------------------+--------------------+-----------+--------------------------+--------------------------+--------------------+----------------+--------+-------+------------------------+----------------+--------------+---------------+---------------+-----------------------+-----------------------+
|customer_id|order_date|item_trans_id|current_date|sold_to_country|product_group|color|brand|order_id|product_class|size|prod_country| fit|price|is_returned|return_date|day_of_year|year|month|week_of_year|      sin_order_date|     cos_order_date|n_item_same_order|n_prod_same_ptype_same_order|n_same_pc_same_order|n_same_pg_same_order|order_value|is_purchase_order_returned|purchase_order_return_date|arr_d

Besides backward looking features, we can also create forward customer features. It only makes sense to do this until the return window closes. 

In [49]:
df = df.withColumn("n_days", F.datediff(F.lit(max_date), F.col('order_date')))

w = psw.partitionBy(["customer_id", "current_date"]).orderBy(F.desc('n_days')).rangeBetween(1, return_window)

df_all = (df
          .withColumn('pur_same_cust_after', F.count("item_trans_id").over(w))
          .withColumn('ret_same_cust_after', F.sum("is_returned").over(w))
          )

For many orders values will be missing for this feature, which has to be imputed.

In [50]:
cust_feat = ['pur_cust_before',
             'ret_cust_before',
             'pur_same_pc_cust_before',
             'ret_same_pc_cust_before',
             'pur_same_cust_after',
             'ret_same_cust_after']

df_all = df_all.fillna(0, cust_feat)          

One final step is to selectively deal with missing values per feature.

In [52]:
# Fill selectively per column
df_all = df_all.na.fill({'prod_country': 'unknown',
                         'fit': 'no_fit',
                         'customer_id': 0,
                         'size': 'no_size',
                         'color': 'unknown'
                        })

That concludes the feature engineering! Now on to modeling.

### Modeling

In [55]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from synapse.ml.lightgbm import LightGBMClassifier, LightGBMClassificationModel
from pyspark.ml.functions import vector_to_array
import pandas as pd

#### Building a Pipeline
We must first indicate which transformations should be applied to which features. Categorical features containing string should first be integer encoded and one-hot encoded after.

In [54]:
# Columns that need integer encoding
str_cols = ['color', 'brand', 'sold_to_country', 'prod_country', 'product_class', 
            'product_group', 'size', 'fit']

outstr_cols = [col+'_ind' for col in str_cols]

# Columns that need one-hot encoding
ohe_cols = outstr_cols + ['month', 'week_of_year']

outohe_cols = [col+'_dummy' for col in ohe_cols]

# Adding numerical features
predictor_cols = outohe_cols + ['sin_order_date', 'cos_order_date', 'price',
                                'order_value', 'n_item_same_order', 
                                'n_same_pc_same_order', 'n_same_pg_same_order', 
                                'rr_group', 'rr_cust'
                                'pur_cust_before', 'ret_cust_before',
                                'pur_same_pc_cust_before', 'ret_same_pc_cust_before',
                                'cust_account_age', 'first_order',
                                'pur_same_cust_after', 'ret_same_cust_after']  

print(predictor_cols)
print(len(predictor_cols))

['color_ind_dummy', 'brand_ind_dummy', 'sold_to_country_ind_dummy', 'prod_country_ind_dummy', 'product_class_ind_dummy', 'product_group_ind_dummy', 'size_ind_dummy', 'fit_ind_dummy', 'month_dummy', 'week_of_year_dummy', 'sin_order_date', 'cos_order_date', 'price', 'order_value', 'n_item_same_order', 'n_same_pc_same_order', 'n_same_pg_same_order', 'rr_group', 'rr_custpur_cust_before', 'ret_cust_before', 'pur_same_pc_cust_before', 'ret_same_pc_cust_before', 'cust_account_age', 'first_order', 'pur_same_cust_after', 'ret_same_cust_after']
26


Next we must instantiate the right transformer classes as well as the estimator and combine these into a pipeline.

In [ ]:
# Create indexer object
indexer = StringIndexer(inputCols = str_cols, outputCols = outstr_cols, handleInvalid = 'keep')

# Create onehotencoder object 
ohe_encoder = OneHotEncoder(inputCols = ohe_cols, outputCols = outohe_cols, handleInvalid = 'keep')

# Create vector assembler object
assembler = VectorAssembler(inputCols = predictor_cols, outputCol ='features', handleInvalid = 'keep')

# Create a LightGBMClassifier object
lgbm = LightGBMClassifier(labelCol = 'is_returned')

# Create Pipeline
pipeline = Pipeline(stages = [indexer, ohe_encoder, assembler, lgbm]) 

We also require an evaluator class to check for model performane.

In [ ]:
# Create an evaluator object
evaluator = BinaryClassificationEvaluator(labelCol = 'is_returned')

#### Splitting the Data
The next step is to split our training and testing data for a backtest according to some business rules. Specifically it must be taken into account that the thirty days before the model training date cannot be used due to size of the return window.

In [ ]:
def generate_split(df, start_date = '2022-08-15', num_days = 30, test_gap = 33, train_length=365):
    
    """ 
    Generates a training and test according to business rules.

    Input:
    - data
    - first day of test set
    - number of days in the test set
    - gap to keep between last day of training data and first day of testing
    - number of historical order dates to train on

    Output:
    - training data
    - testing data
    """
    
    testset_start_date = F.to_date(F.lit(start_date))
    testset_end_date = F.date_add(F.lit(testset_start_date), num_days)
    
    model_train_date = F.date_sub(F.lit(testset_start_date), 1)

    trainset_end_date = F.date_sub(F.lit(testset_start_date), test_gap)
    

    df_copy = df.alias('df_copy')

    df_train =  df_copy.filter(((df_copy.order_date >= F.date_sub(F.lit(trainset_end_date), train_length)) & 
                                (df_copy.order_date <= trainset_end_date))  
                                  )

    # If the test gap is shorter than the return window, the labels must be adjusted
    # in case a return was not known yet on the model training date
    df_train = df_train.withColumn('is_returned', F.when(df_train.return_date > model_train_date, 0)
                                                   .otherwise(df_train.is_returned))
    
    df_test = df_copy.filter((df_copy.current_date >= testset_start_date)\
                           & (df_copy.current_date <= testset_end_date))
    
    
    return df_train, df_test

#### Evaluation Function
Another function is required to make NMV predictions, aggregate by order date and calculate absolute percent error. Predictions are then aggregated to the prediction date level to calculate the mean absolute percent error for every prediction date.

In [ ]:
def vec_to_arr(df, new_col = "prob", old_col = "probability"):
  """ Helper function for evaluate_model""" 
    df = df.withColumn(new_col, vector_to_array(old_col))
    return df

In [ ]:
def evaluate_model(model, data, country=False):
                   
    """ Calculates total sales, actual nmv (net merchandise value) and predicted nmv for a date or list of dates.
        Subsequently calculates the absolute percentage error for the nmv value prediction for each date.
        Finally calculates the mean absolute percentage error and the mean bias for all the dates predicted on for each current date.
        
        Input: 
            - a binary classification model
            - an exploded test dataset with a current_date and price columns
            - display results on country level or not

        Output:
            - dataframe detailing total sales, actual nmv, predicted nmv, absolute percent error
            - MAPE (mean absolute percent error)
            - bias
            
            Output is returned as a tuple."""
    
    # Make predictions with model
    df_pred = model.transform(data)
    df_pred = vec_to_arr(df_pred)
    
    # Calculate item-level predicted- and actual NMV
    df_pred = df_pred.withColumn('nmv_pred', (1 - df_pred['prob'][1]) * df_pred['item_value_eur'])
    df_pred = df_pred.withColumn('nmv_act', F.col("item_value_eur") * (1 - F.col("is_returned")))
    
    
    if country == False:
            df_res = (df_pred.groupBy(["order_date", "current_date"])
                    .agg(F.round(F.sum("price")).alias('sales_value'),
                         F.round(F.sum("nmv_act")).alias('nmv_act'),
                         F.round(F.sum("nmv_pred")).alias('nmv_pred')
                        ))
            
    if country == True:
            df_res = (df_pred.groupBy(["order_date", "current_date", "sold_to_country"])
                    .agg(F.round(F.sum("price")).alias('sales_value'),
                         F.round(F.sum("nmv_act")).alias('nmv_act'),
                         F.round(F.sum("nmv_pred")).alias('nmv_pred')
                        ))
    
    df_res = df_res.withColumn('APE', F.round(F.abs(df_res.nmv_act - df_res.nmv_pred) / df_res.nmv_act *100, 1))
   
    if country == False:
            df_per_current_date = (df_res.groupBy('current_date')
                              .agg(F.round(F.mean("APE"), 3).alias('MAPE'),
                                  F.round(F.sum("nmv_act")).alias('sum_nmv_act'),
                                  F.round(F.sum("nmv_pred")).alias('sum_nmv_pred'),
                                  )) 
    if country == True:
            df_per_current_date = (df_res.groupBy(['current_date', 'sold_to_country'])
                              .agg(F.round(F.mean("APE"), 3).alias('MAPE'),
                                  F.round(F.sum("nmv_act")).alias('sum_nmv_act'),
                                  F.round(F.sum("nmv_pred")).alias('sum_nmv_pred'),
                                  )) 
                                
    mape = df_per_current_date.agg({'MAPE':'mean'})
    mean_bias = df_per_current_date.agg({'bias':'mean'})
    
    return df_pred, df_res, df_per_current_date.sort('current_date'), mape

#### Training the model
From here on, training and evaluating the model is straightforward. We simply split the data according to our previously defined function and proceed to training.

In [ ]:
df_train, df_test = generate_split()

In [ ]:
model = pipeline.fit(df_train)

In [ ]:
df_pred, df_res, df_current_date, mape = evaluate_model(model, df_test)

#### Checking Performance
Now we can analyze the results that we have obtained.

In [ ]:
df_res.show()

In [ ]:
df_current_date.show(31)

As a final step it is also possible to check the importance of each feature. The spark LightGBM module offers two methods for this: split and gain. 

*   split: the number of nodes that were split on this feature
*   gain: total information gain for all splits over this feature



In [ ]:
va = model.stages[-2]
lgbm = model.stages[-1]

importances = dict(zip(va.getInputCols(), lgbm.getFeatureImportances(importance_type = 'gain')))

importances = pd.DataFrame.from_dict(importances, orient='index', columns = ['importance'])
importances = importances.sort_values('importance', ascending = False)

importances.plot.barh()